# 🚀 RAG PDF Analyzer - LOCAL EMBEDDINGS

**Works with exhausted API quota!**

- ✅ Local embeddings (NO API calls for upload)
- ✅ Gemini for Q&A only (1 API call per question)
- ✅ Fast uploads (no network delay)
- ✅ Works offline for PDF processing

## 📦 Step 1: Install Packages

In [ ]:
!pip install -q sentence-transformers langchain langchain-community langchain-google-genai langchain-classic google-generativeai faiss-cpu PyPDF2 langchain-text-splitters langchain-core torch

## 🔑 Step 2: Set API Key (Only for Q&A)

In [ ]:
import os
os.environ['GOOGLE_API_KEY'] = 'YOUR_API_KEY'   
print("✅ API Key set (only used for questions)!"

✅ API Key set (only used for questions)!


## 📚 Step 3: Import Libraries

In [35]:
import PyPDF2, warnings, uuid
from datetime import datetime
from typing import Dict, List
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_classic.chains import RetrievalQA
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer

warnings.filterwarnings('ignore')
print("✅ Imports ready!")

✅ Imports ready!


## 🛠️ Step 4: Local Embeddings Class

In [36]:
class LocalEmbeddings(Embeddings):
    """Local embeddings using sentence-transformers (runs on your PC)"""
    
    def __init__(self):
        print("📥 Loading local embedding model (first time ~100MB download)...")
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        print("✅ Local model loaded!\n")
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embed multiple texts"""
        return self.model.encode(texts).tolist()
    
    def embed_query(self, text: str) -> List[float]:
        """Embed single query"""
        return self.model.encode([text])[0].tolist()

print("✅ Local embeddings class ready!")

✅ Local embeddings class ready!


## 🤖 Step 5: RAG Engine with Local Embeddings

In [37]:
class RAGEngine:
    def __init__(self):
        print("🔧 Initializing RAG with LOCAL embeddings...")
        
        # LOCAL embeddings (NO API calls!)
        self.embeddings = LocalEmbeddings()
        
        # Gemini for Q&A only
        self.llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.3)
        
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            separators=["\n\n", "\n", ". ", " "]
        )
        
        self.vector_store = None
        self.documents_metadata = {}
        print("✅ Ready! Upload uses NO API calls!\n")
    
    def extract_text_from_pdf(self, pdf_path: str) -> str:
        text_parts = []
        with open(pdf_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            print(f"📖 Reading {len(reader.pages)} pages...\n")
            for i, page in enumerate(reader.pages, 1):
                try:
                    text = page.extract_text()
                    if text and text.strip():
                        text_parts.append(text)
                        print(f"   ✅ Page {i}")
                except:
                    print(f"   ❌ Page {i}")
        full = "\n\n".join(text_parts)
        print(f"\n✅ Extracted {len(full)} chars\n")
        return full
    
    def add_document(self, pdf_path: str, filename: str = None) -> str:
        print(f"\n{'='*50}")
        print(f"📄 PROCESSING: {filename or pdf_path}")
        print(f"{'='*50}\n")
        
        text = self.extract_text_from_pdf(pdf_path)
        doc_id = str(uuid.uuid4())
        chunks = self.text_splitter.split_text(text)
        
        print(f"📊 Creating {len(chunks)} chunks...")
        print(f"💻 Using LOCAL embeddings (no API calls!)\n")
        
        # Prepare metadata
        metas = [{"document_id": doc_id, "chunk_id": i, "filename": filename or pdf_path} 
                for i in range(len(chunks))]
        
        # Create embeddings locally (FAST!)
        if not self.vector_store:
            self.vector_store = FAISS.from_texts(chunks, self.embeddings, metas)
        else:
            new = FAISS.from_texts(chunks, self.embeddings, metas)
            self.vector_store.merge_from(new)
        
        self.documents_metadata[doc_id] = {
            "id": doc_id,
            "filename": filename or pdf_path,
            "chunks_count": len(chunks),
            "created_at": datetime.now().isoformat()
        }
        
        print(f"{'='*50}")
        print(f"✅ DONE! {len(chunks)} chunks embedded locally")
        print(f"{'='*50}\n")
        return doc_id
    
    def query(self, question: str, k: int = 3) -> Dict:
        if not self.vector_store:
            return {"error": "No documents! Upload a PDF first."}
        
        print(f"\n❓ {question}\n")
        print("🔍 Searching locally...")
        
        retriever = self.vector_store.as_retriever(search_kwargs={"k": k})
        chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True
        )
        
        print("🤖 Asking Gemini (1 API call)...\n")
        result = chain({"query": question})
        
        return {
            "answer": result["result"],
            "sources": len(result.get("source_documents", []))
        }

print("✅ RAG Engine ready!")

✅ RAG Engine ready!


## 🎯 Step 6: Initialize Engine

In [38]:
rag = RAGEngine()

🔧 Initializing RAG with LOCAL embeddings...
📥 Loading local embedding model (first time ~100MB download)...
✅ Local model loaded!

✅ Ready! Upload uses NO API calls!



## 📄 Step 7: Upload PDF (NO API CALLS!)

In [39]:
pdf_path = "e.pdf"  # Change to your file

try:
    doc_id = rag.add_document(pdf_path, "My Document")
    print("🎉 Success! No API calls used!")
except Exception as e:
    print(f"❌ Error: {e}")


📄 PROCESSING: My Document

📖 Reading 8 pages...

   ✅ Page 1
   ✅ Page 2
   ✅ Page 3
   ✅ Page 4
   ✅ Page 5
   ✅ Page 6
   ✅ Page 7
   ✅ Page 8

✅ Extracted 22834 chars

📊 Creating 29 chunks...
💻 Using LOCAL embeddings (no API calls!)

✅ DONE! 29 chunks embedded locally

🎉 Success! No API calls used!


## 💬 Step 8: Ask Questions (Uses Gemini)

In [40]:
question = "What is the main topic of this document?"

result = rag.query(question)

if "error" not in result:
    print(f"💡 Answer:\n{result['answer']}")
    print(f"\n📚 Used {result['sources']} sources")
else:
    print(result['error'])


❓ What is the main topic of this document?

🔍 Searching locally...
🤖 Asking Gemini (1 API call)...

💡 Answer:
The main topic of this document is the **impact and role of print culture and the printing press**, particularly in the context of the **Reformation Movement** and the **transformation of European society**.

📚 Used 3 sources


## 📝 Step 9: Ask More Questions

In [42]:
# Ask another question
result = rag.query("Summarize the key points")
if "error" not in result:
    print(f"💡 {result['answer']}")
    print(f"\n📚 Used {result['sources']} sources")
else:
    print(result['error'])


❓ Summarize the key points

🔍 Searching locally...
🤖 Asking Gemini (1 API call)...

💡 Here are the key points from the provided text:

*   **The Silk Routes:** Ancient trade routes connecting China, Central Asia, India, and Europe, facilitating the exchange of goods (like silk, spices, textiles) and the spread of culture, religion, and ideas.
*   **Food Travels and Globalization:** The exchange of crops (e.g., potatoes, maize from America to other continents) transformed diets, improved nutrition, led to population growth, and connected societies globally.
*   **Impact of the Industrial Revolution:** Started in Britain, it increased production and demand for raw materials, with colonies supplying these materials and serving as markets. Improved transport (railways, steamships) and communication (telegraph) linked global markets, creating a worldwide economic network controlled by industrial powers.
*   **Causes of the Great Depression (1929-1934):** Overproduction and falling prices i